## 14.2 矩阵分解
---
任何一个m*n阶矩阵$Data_{m\times n}$都可以分解成：
$$Data_{m\times n}=U_{m\times m}\Sigma _{m\times n}V_{n\times n}^T$$
其中，$U,V^T$是单位正交矩阵，$\Sigma$ 只有对角元素，其余元素均为零。且其对角元素从大到小排列，这些对角元素称为奇异值。

## 14.3 利用Python实现SVD
基于np.linalg.svd函数

In [1]:
import numpy as np


U,Sigma,VT=np.linalg.svd(np.array([[1,1],[7,7]]))
print('U: \n', U, 'Sigma: \n', Sigma, 'VT: \n', VT)    # 但需要注意到，这里的\Sigma是以一个列向量的形式返回的。

U:  [[-0.14142136 -0.98994949]
 [-0.98994949  0.14142136]] Sigma:  [10.  0.] VT:  [[-0.70710678 -0.70710678]
 [-0.70710678  0.70710678]]


## 14.4 基于协同过滤的推荐引擎
---
一个例子，可以通过计算某一个新电影X和用户看过的喜欢的电影A之间的**相似度**，如果该值较高的话，则可认为用户也喜欢电影X,进而将该电影推荐给用户。

### 14.4.1 相似度计算
常见的相似度计算方式有三种：
- 计算两个向量之间的欧氏距离
- 皮尔逊相关系数
- 余弦相似度
---
#### 程序清单 14-1 相似度的计算 

In [2]:
import numpy.linalg as la 


def eulidSim(inA,inB):
    return 1/(1+la.norm(inA-inB))    # 范数


def pearsSim(inA,inB):
    if len(inA)<3:return 1.0
    return 0.5+0.5*np.corrcoef(inA,inB,rowvar=0)[0][1]


def cosSim(inA,inB):
    num=inA.T*inB
    denom=la.norm(inA)+la.norm(inB)
    return 0.5+0.5*(num/denom)


# 数据加载的函数
def loadData():
    return [
        [1,1,1,0,0],
        [2,2,2,0,0],
        [1,1,1,0,0],
        [5,5,5,0,0],
        [1,1,0,2,2],
        [0,0,0,3,3],
        [0,0,0,1,1]
    ]
    
    
myMat=np.mat(loadData())
print('欧几里得相似度: ',eulidSim(myMat[:,0],myMat[:,4]))
print('余弦相似度: ',cosSim(myMat[:,0],myMat[:,4]))
print('皮尔逊相似度: ',pearsSim(myMat[:,0],myMat[:,4]))

欧几里得相似度:  0.13367660240019172
余弦相似度:  [[0.60639983]]
皮尔逊相似度:  0.23768619407595826


## 14.5 示例：餐馆菜肴推荐引擎
### 14.5.1 推荐没尝过的菜肴
#### 程序清单14-2 基于物品相似度的推荐引擎

In [11]:
def standEst(dataMat,user,simMeas,item):
    """
    估计user对商品item的评分。
    user:用户编号
    item:未评分的物品编号
    """
    n=dataMat.shape[1]
    simTotal=0.0
    ratSimTotal=0.0
    for j in range(n):
        userRating=dataMat[user,j]    # 当前用户对j商品的评分
        if userRating==0:continue
        overlap=np.nonzero(np.logical_and(dataMat[:,j].A>0,dataMat[:,item].A>0))[0]    # 找出同时评价j和item的用户
        if len(overlap)==0:
            similarity=0
        else:
            similarity=simMeas(dataMat[overlap,j],dataMat[overlap,item])
            simTotal+=similarity
            ratSimTotal+=similarity*userRating    # 加权相似度（权中为用户评分）
    if simTotal==0:return 0
    return ratSimTotal/simTotal    # 归一化，使得最后的评分值在0-5之间


def recommend(dataMat,user,N=3,simMeas=cosSim,estMethod=standEst):
    """对user所有没评过分的商品都进行评分的估计"""
    unratedItems=np.nonzero(dataMat[user,:].A==0)[1]    # 注意这里输出的是一个长度为2的元组（由二维数组导致），所以取第二个
    if len(unratedItems)==0:
        print('You have rated everything!!')
    itemScores=[]
    for item in unratedItems:    # 在所有未评分的商品中遍历
        estimatedScore=estMethod(dataMat,user,simMeas,item)    # 估计评分
        itemScores.append((item,estimatedScore))
    return sorted(itemScores,key=lambda x:x[1],reverse=True)[:N]    # 按照estimatedScore进行从大到小排序


# 对myMat做一些简单的变动（可能是为了效果更好？）
myMat[0,1]=myMat[0,0]=myMat[1,0]=myMat[2,0]=4
myMat[3,3]=2
recommend(myMat,2)

[(2, matrix([[3.08266119]])), (1, matrix([[2.44390502]]))]

In [7]:
myMat

matrix([[4, 4, 1, 0, 0],
        [4, 2, 2, 0, 0],
        [4, 1, 1, 0, 0],
        [5, 5, 5, 2, 0],
        [1, 1, 0, 2, 2],
        [0, 0, 0, 3, 3],
        [0, 0, 0, 1, 1]])

In [12]:
myMat=[
    [4,4,0,2,2],
    [4,0,0,3,3],
    [4,0,0,1,1],
    [1,1,1,2,0],
    [2,2,2,0,0],
    [1,1,1,0,0],
    [5,5,5,0,0],
]
myMat=np.mat(myMat)
recommend(myMat,2)

[(2, matrix([[3.08266119]])), (1, matrix([[2.44390502]]))]

### 14.5.2 利用SVD提高推荐的效果
---
能量的概念：
矩阵A的前n个奇异值的平方和，叫做这n个奇异值的能量，所有奇异值的平方和，称为总能量。

In [13]:
def loadExData2():
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]
    
    
U,Sigma,VT=np.linalg.svd(np.mat(loadExData2()))
Sigma

array([15.77075346, 11.40670395, 11.03044558,  4.84639758,  3.09292055,
        2.58097379,  1.00413543,  0.72817072,  0.43800353,  0.22082113,
        0.07367823])

#### 我们想要看一下，用多少个奇异值，能够达到总能量的90%

In [14]:
Sig2=Sigma**2

i=0
while (Sig2[:i]**2).sum()<0.9*Sig2.sum():
    i+=1
i
# i=1,这意味着只需使用前2个奇异值的能量即可达到总能量的90%.

1